In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
warnings.filterwarnings("ignore")

    
##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._toolbox as sf


In [66]:
##  SETUP DIRECTORIES

dir_cur = pathlib.Path(os.getcwd())
dir_data = dir_cur.joinpath("data")
dir_transformations = dir_cur.joinpath("transformations")


##  SETUP FILES

fp_data_base = dir_data.joinpath("iran_inputs_20241031.csv")


##  SETUP SOME SISEPUEDE STUFF

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)

In [22]:
##  BUILD BASE INPUTS

df_inputs_raw = pd.read_csv(fp_data_base)

# pull example data to fill in gaps
examples = SISEPUEDEExamples()
df_inputs_example = examples.input_data_frame

###   Review fields that are different

In [23]:
[
    x for x in df_inputs_example.columns if x not in df_inputs_raw.columns
]

['region',
 'ef_enfu_combustion_tonne_co2_per_tj_fuel_water',
 'ef_enfu_stationary_combustion_tonne_ch4_per_tj_fuel_water',
 'ef_enfu_stationary_combustion_tonne_n2o_per_tj_fuel_water',
 'strategy_id',
 'cost_enfu_fuel_ammonia_usd_per_tonne',
 'ef_enfu_combustion_tonne_co2_per_tj_fuel_ammonia',
 'ef_trns_mobile_combustion_water_borne_kg_n2o_per_tj_ammonia',
 'ef_enfu_stationary_combustion_tonne_ch4_per_tj_fuel_ammonia',
 'ef_enfu_stationary_combustion_tonne_n2o_per_tj_fuel_ammonia',
 'exports_enfu_pj_fuel_ammonia',
 'frac_enfu_fuel_demand_imported_pj_fuel_ammonia',
 'nemomod_entc_capital_cost_fp_ammonia_production_mm_usd_per_gw',
 'nemomod_entc_capital_cost_fp_hydrogen_reformation_ccs_mm_usd_per_gw',
 'nemomod_entc_emissions_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_tonne_co2_per_tj',
 'nemomod_entc_emissions_activity_ratio_scalar_fp_hydrogen_reformation_ccs_co2',
 'nemomod_entc_fixed_cost_fp_ammonia_production_mm_usd_per_gw',
 'nemomod_entc_fixed_cost_fp_hydrogen_refo

In [17]:
##  SOME MANUAL FIXES FOR THE MOMENT

df_inputs_example["exports_enfu_pj_fuel_ammonia"] = 0


###   Build the file here

- Drop unecessary fields
- Use systematic keys/index fields
- add region name

In [56]:
fields_drop_example = [
    x for x in [regions.key, matt.dim_strategy_id]
    if x in df_inputs_example
]

fields_drop_raw = [
    x for x in ["iso_code3", "Year"]
    if x in df_inputs_example
]

df_inputs = sf.match_df_to_target_df(
    df_inputs_example
    .drop(
        columns = fields_drop_example,
    ),
    df_inputs_raw
    .drop(
        columns = fields_drop_raw,
    ),
    [time_periods.field_time_period],
    overwrite_only = False,
)

df_inputs[regions.key] = "iran"

##  Let's try building transformations using this


In [60]:
transformers = trf.transformers.Transformers(
    {},
    df_input = df_inputs,
)


####  Instantiate some transformations

In [67]:
# set an ouput path and instantiate
trf.instantiate_default_strategy_directory(
    transformers,
    dir_transformations,
)


In [ ]:
# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
    dir_transformations,
    transformers = transformers,
)
transformations.attribute_transformation.table




In [ ]:
dir_transformations_out = dir_